In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from google.colab import files

uploaded = files.upload()

Saving 202501_clean2.xlsx to 202501_clean2.xlsx


In [4]:
df = pd.read_excel('202501_clean2.xlsx')

In [5]:
# 금속 농도 컬럼 추출
metal_columns = [col for col in df.columns if '(ng/m3)' in col or '(ug/m3)' in col]
df_metals = df[metal_columns]

In [6]:
# 음수값을 NaN 처리
df[metal_columns] = df[metal_columns].applymap(lambda x: np.nan if x < 0 else x)

/tmp/ipython-input-4222275292.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[metal_columns] = df[metal_columns].applymap(lambda x: np.nan if x < 0 else x)


In [7]:
# 결과 저장 데이터프레임 생성
result_table = pd.DataFrame(index=metal_columns, columns=metal_columns)

In [11]:
# Pearson 상관계수 계산
for i, col1 in enumerate(metal_columns):
    for j, col2 in enumerate(metal_columns):
        # 상삼각은 비움(중복 제거)
        if j > i:
            continue
        # 대각선은 항상 1 표시
        if i == j:
            result_table.iat[i, j] = '1'
            continue

        # 1:1 매칭
        pair = df[[col1, col2]].apply(pd.to_numeric, errors='coerce').dropna()
        if len(pair) < 3 or pair[col1].nunique() < 2 or pair[col2].nunique() < 2:
            result_table.iat[i, j] = np.nan
            continue

        # 유의수준 확인
        r, p = pearsonr(pair[col1].values, pair[col2].values)
        symbol = '**' if p < 0.01 else ('*' if p < 0.05 else '')
        result_table.iat[i, j] = f'{r:.3f}{symbol}'

# 결과 확인
result_table.iloc[:10, :10]

,Conc(ug/m3),Cr(ng/m3),Co(ng/m3),Ni(ng/m3),As(ng/m3),Cd(ng/m3),Sb(ng/m3),Pb(ng/m3)
Conc(ug/m3),1,,,,,,,
Cr(ng/m3),0.035,1,,,,,,
Co(ng/m3),0.668**,0.302**,1,,,,,
Ni(ng/m3),-0.320**,0.929**,0.001,1,,,,
As(ng/m3),NaN,NaN,NaN,NaN,1,,,
Cd(ng/m3),0.081,0.892**,0.379**,0.956**,NaN,1,,
Sb(ng/m3),-0.002,0.213**,0.139*,0.183*,NaN,0.290**,1,
Pb(ng/m3),0.607**,0.064,0.508**,-0.236**,NaN,0.107*,0.008,1


In [14]:
# 엑셀 저장 코드
output_path = "상관관계분석표.xlsx"
result_table.to_excel(output_path, na_rep="NaN")  # NaN을 텍스트 'NaN'으로 저장

files.download("상관관계분석표.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>